Multiple Linear Regression

importing needed packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline

Downloading data

To download the data,we will use !wget to download it from IBM Object Storage

In [4]:
!wget -O FuelConsumption.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv

--2021-03-11 10:25:03--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72629 (71K) [text/csv]
Saving to: ‘FuelConsumption.csv’

FuelConsumption.csv 100%[===================>]  70.93K  --.-KB/s    in 0.06s   

2021-03-11 10:25:03 (1.21 MB/s) - ‘FuelConsumption.csv’ saved [72629/72629]



Understanding the Data

FuelConsumption.csv

We have downloaded a fuel consumption dataset,FuelConsumption.csv which contains model-specific fuel consumption ratings and estimated carbon dioxide emissions
for new light duty vehicles for retail sale in Canada

MODEL YEAR eg:2004

MAKE e.g. Acura

MODEL e.g ILX

VEHICLE CLASS e.g SUV

ENGINE SIZE e.g 4.7

CYLINDERS e.g 6

TRANSMISSION e.g A6

FUELTYPE e.g z

FUEL CONSUMPTION in CITY(L/100 KM) e.g. 9.9

FUEL CONSUMPTION in HWY(L/100KM) e.g. 8.9

FUEL CONSUMPTION COMB(L/100KM) e.g.9.2

CO2 EMISSIONS(g/km) e.g 182--> low --> 0

Reading the data in